<a href="https://colab.research.google.com/github/tomdaler/Machine-Learning/blob/main/Optimization/Optimal_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('winequality-red.csv')


In [ ]:
df['quality']= df['quality']-3

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

scaled_features = df.copy()

col_names = ['fixed acidity','free sulfur dioxide','total sulfur dioxide','alcohol','pH','residual sugar']
features = scaled_features[col_names]
df_new = scale.fit_transform(features)

scaled_features[col_names] = df_new

In [ ]:
X = scaled_features.drop(['quality'],axis=1)
y = scaled_features['quality']
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=2)

In [ ]:
X=X.drop(['alcohol', 'sulphates','pH','density','total sulfur dioxide'], axis=1)

These fields are not important
- alcohol
- solphates
- pH
- density
- total sulfur dioxide

Check among models

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from sklearn.metrics import accuracy_score

#Eliminar warnings
import warnings
warnings.simplefilter("ignore")

classifier = {
        'KNeighbors': KNeighborsClassifier(),
        'LinearSCV': LinearSVC(),
        'SVC': SVC(),
        'SGDC': SGDClassifier(),
        'DecisionTree': DecisionTreeClassifier(),
        'RandomForestClassifier': RandomForestClassifier(),
        'LogisticRegression': LogisticRegression(),
        'svm': SVC(decision_function_shape='ovo')
    }

for name, estimator in classifier.items():
    bag_class = BaggingClassifier(base_estimator=estimator, n_estimators=5).fit(train_X, train_y)
    bag_pred = bag_class.predict(test_X)
    print('Accuracy Bagging with {}:'.format(name), accuracy_score(bag_pred, test_y))
    print('')


Accuracy Bagging with KNeighbors: 0.5416666666666666

Accuracy Bagging with LinearSCV: 0.54375

Accuracy Bagging with SVC: 0.5625

Accuracy Bagging with SGDC: 0.55

Accuracy Bagging with DecisionTree: 0.59375

Accuracy Bagging with RandomForestClassifier: 0.63125

Accuracy Bagging with LogisticRegression: 0.5791666666666667

Accuracy Bagging with svm: 0.5583333333333333



In [ ]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [100,120, 140],
            'max_samples': [0.7,0.9] 
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,10,20]
        }
    }
}

scores = []

from sklearn.model_selection import GridSearchCV

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.494683,"{'C': 20, 'kernel': 'linear'}"
1,random_forest,0.479048,"{'max_samples': 0.7, 'n_estimators': 100}"
2,logistic_regression,0.490917,{'C': 20}


Bagging Cross Val

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.7, 
    oob_score=True,
    random_state=0
)
bag_model.fit(train_X, train_y)
bag_model.oob_score_

0.6791778373547811